In [ ]:
import copy
import numpy as np

# Constants
INPUT_FILE_NAME = 'input.txt'
OUTPUT_FILE_NAME = 'output.txt'
STEP_NUMBER_FILE_NAME = 'step_num.txt'
BOARD_SIZE = 5
UNOCCUPIED = 0
BLACK = 1
WHITE = 2
KOMI = 2.5
X_CHANGES = [1, 0, -1, 0]
Y_CHANGES = [0, 1, 0, -1]

# Main class for the Go player
class GoPlayer:
    def __init__(self, player_color, previous_board, current_board):
        self.player_color = player_color
        self.opponent_color = BLACK if player_color == WHITE else WHITE
        self.previous_board = previous_board
        self.current_board = current_board
        self.transposition_table = {}

    def alpha_beta_search(self, depth, branching_factor, step_number):
        # Initialize alpha and beta values for alpha-beta pruning
        alpha = -np.inf
        beta = np.inf
        is_repetition = self.check_repetition(self.current_board)

        # Perform alpha-beta search
        best_move, best_value = self.max_value(self.current_board, self.player_color, depth, 0, branching_factor, alpha, beta, None, step_number, False)

        # Write the best move to output
        if best_move and not is_repetition:
            write_output(best_move)
        else:
            write_output((-1, -1))

    def max_value(self, board, player_color, depth, current_depth, branching_factor, alpha, beta, last_move, step_number, is_second_pass):
        # Terminate if the depth limit is reached or maximum steps are played
        if depth == current_depth or step_number + current_depth == 24:
            return self.evaluate_board(board, player_color)
        if is_second_pass:
            return self.evaluate_board(board, player_color)

        is_second_pass = False
        max_move_value = -np.inf
        max_move = None
        valid_moves = self.find_valid_moves(board, player_color)
        valid_moves.append((-1, -1))
        if last_move == (-1, -1):
            is_second_pass = True

        for move in valid_moves[:branching_factor]:
            opponent_color = self.get_opponent_color(player_color)
            if move == (-1, -1):
                new_board = copy.deepcopy(board)
            else:
                new_board = self.make_move(board, player_color, move)
            min_move_value = self.min_value(new_board, opponent_color, depth, current_depth + 1, branching_factor, alpha, beta, move, step_number, is_second_pass)
            if max_move_value < min_move_value:
                max_move_value = min_move_value
                max_move = move
            if max_move_value >= beta:
                if current_depth == 0:
                    return max_move, max_move_value
                else:
                    return max_move_value
            alpha = max(alpha, max_move_value)

        if current_depth == 0:
            return max_move, max_move_value
        else:
            return max_move_value

    def min_value(self, board, player_color, depth, current_depth, branching_factor, alpha, beta, last_move, step_number, is_second_pass):
        # Terminate if the depth limit is reached or maximum steps are played
        if depth == current_depth:
            return self.evaluate_board(board, player_color)
        if step_number + current_depth == 24 or is_second_pass:
            return self.evaluate_board(board, self.player_color)

        is_second_pass = False
        min_move_value = np.inf
        valid_moves = self.find_valid_moves(board, player_color)
        valid_moves.append((-1, -1))
        if last_move == (-1, -1):
            is_second_pass = True

        for move in valid_moves[:branching_factor]:
            opponent_color = self.get_opponent_color(player_color)
            if move == (-1, -1):
                new_board = copy.deepcopy(board)
            else:
                new_board = self.make_move(board, player_color, move)
            max_move_value = self.max_value(new_board, opponent_color, depth, current_depth + 1, branching_factor, alpha, beta, move, step_number, is_second_pass)
            if max_move_value < min_move_value:
                min_move_value = max_move_value
            if min_move_value <= alpha:
                return min_move_value
            beta = min(beta, min_move_value)

        return min_move_value

    def evaluate_board(self, board, player_color):
        # Heuristic evaluation of the board state
        opponent_color = self.get_opponent_color(player_color)
        player_count = 0
        player_liberty = set()
        opponent_count = 0
        opponent_liberty = set()

        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if board[i][j] == player_color:
                    player_count += 1
                elif board[i][j] == opponent_color:
                    opponent_count += 1
                else:
                    for k in range(len(X_CHANGES)):
                        new_i = i + X_CHANGES[k]
                        new_j = j + Y_CHANGES[k]
                        if 0 <= new_i < BOARD_SIZE and 0 <= new_j < BOARD_SIZE:
                            if board[new_i][new_j] == player_color:
                                player_liberty.add((i, j))
                            elif board[new_i][new_j] == opponent_color:
                                opponent_liberty.add((i, j))

        player_edge_count = 0
        opponent_edge_count = 0
        for j in range(BOARD_SIZE):
            if board[0][j] == player_color or board[BOARD_SIZE - 1][j] == player_color:
                player_edge_count += 1
            if board[0][j] == opponent_color or board[BOARD_SIZE - 1][j] == opponent_color:
                opponent_edge_count += 1

        for j in range(1, BOARD_SIZE - 1):
            if board[j][0] == player_color or board[j][BOARD_SIZE - 1] == player_color:
                player_edge_count += 1
            if board[j][0] == opponent_color or board[j][BOARD_SIZE - 1] == opponent_color:
                opponent_edge_count += 1

        center_unoccupied_count = 0
        for i in range(1, BOARD_SIZE - 1):
            for j in range(1, BOARD_SIZE - 1):
                if board[i][j] == UNOCCUPIED:
                    center_unoccupied_count += 1

        score = min(max((len(player_liberty) - len(opponent_liberty)), -8), 8) + (
                -4 * self.calculate_euler_number(board, player_color)) + (
                        5 * (player_count - opponent_count)) - (9 * player_edge_count * (center_unoccupied_count / 9))
        if self.player_color == WHITE:
            score += KOMI

        return score

    def make_move(self, board, player_color, move):
        new_board = copy.deepcopy(board)
        new_board[move[0]][move[1]] = player_color
        for k in range(len(X_CHANGES)):
            new_i = move[0] + X_CHANGES[k]
            new_j = move[1] + Y_CHANGES[k]
            if 0 <= new_i < BOARD_SIZE and 0 <= new_j < BOARD_SIZE:
                opponent_color = self.get_opponent_color(player_color)
                if new_board[new_i][new_j] == opponent_color:
                    if not self.check_for_liberty(new_board, new_i, new_j, opponent_color):
                        self.remove_group(new_board, new_i, new_j, opponent_color)
        return new_board

    def calculate_euler_number(self, board, player_color):
        opponent_color = self.get_opponent_color(player_color)
        padded_board = np.zeros((BOARD_SIZE + 2, BOARD_SIZE + 2), dtype=int)
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                padded_board[i + 1][j + 1] = board[i][j]

        q1_player = 0
        q2_player = 0
        q3_player = 0
        q1_opponent = 0
        q2_opponent = 0
        q3_opponent = 0

        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                sub_board = padded_board[i: i + 2, j: j + 2]
                q1_player += self.count_q1(sub_board, player_color)
                q2_player += self.count_q2(sub_board, player_color)
                q3_player += self.count_q3(sub_board, player_color)
                q1_opponent += self.count_q1(sub_board, opponent_color)
                q2_opponent += self.count_q2(sub_board, opponent_color)
                q3_opponent += self.count_q3(sub_board, opponent_color)

        return (q1_player - q3_player + 2 * q2_player - (q1_opponent - q3_opponent + 2 * q2_opponent)) / 4

    def count_q1(self, sub_board, color):
        if ((sub_board[0][0] == color and sub_board[0][1] != color and sub_board[1][0] != color and sub_board[1][1] != color)
                or (sub_board[0][0] != color and sub_board[0][1] == color and sub_board[1][0] != color and sub_board[1][1] != color)
                or (sub_board[0][0] != color and sub_board[0][1] != color and sub_board[1][0] == color and sub_board[1][1] != color)
                or (sub_board[0][0] != color and sub_board[0][1] != color and sub_board[1][0] != color and sub_board[1][1] == color)):
            return 1
        return 0

    def count_q2(self, sub_board, color):
        if ((sub_board[0][0] == color and sub_board[0][1] != color and sub_board[1][0] != color and sub_board[1][1] == color)
                or (sub_board[0][0] != color and sub_board[0][1] == color and sub_board[1][0] == color and sub_board[1][1] != color)):
            return 1
        return 0

    def count_q3(self, sub_board, color):
        if ((sub_board[0][0] == color and sub_board[0][1] == color and sub_board[1][0] == color and sub_board[1][1] != color)
                or (sub_board[0][0] != color and sub_board[0][1] == color and sub_board[1][0] == color and sub_board[1][1] == color)
                or (sub_board[0][0] == color and sub_board[0][1] != color and sub_board[1][0] == color and sub_board[1][1] == color)
                or (sub_board[0][0] != color and sub_board[0][1] == color and sub_board[1][0] == color and sub_board[1][1] == color)):
            return 1
        return 0

    def find_valid_moves(self, board, player_color):
        valid_moves = {'3side': [], '1capturing': [], '2regular': []}
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if board[i][j] == UNOCCUPIED:
                    if self.check_for_liberty(board, i, j, player_color):
                        if not self.check_for_ko(i, j):
                            if i == 0 or j == 0 or i == BOARD_SIZE - 1 or j == BOARD_SIZE - 1:
                                valid_moves['3side'].append((i, j))
                            else:
                                valid_moves['2regular'].append((i, j))
                    else:
                        for k in range(len(X_CHANGES)):
                            new_i = i + X_CHANGES[k]
                            new_j = j + Y_CHANGES[k]
                            if 0 <= new_i < BOARD_SIZE and 0 <= new_j < BOARD_SIZE:
                                opponent_color = self.get_opponent_color(player_color)
                                if board[new_i][new_j] == opponent_color:
                                    new_board = copy.deepcopy(board)
                                    new_board[i][j] = player_color
                                    if not self.check_for_liberty(new_board, new_i, new_j, opponent_color):
                                        if not self.check_for_ko(i, j):
                                            valid_moves['1capturing'].append((i, j))
                                        break

        valid_moves_list = []
        for move in valid_moves['1capturing']:
            valid_moves_list.append(move)
        for move in valid_moves['2regular']:
            valid_moves_list.append(move)
        for move in valid_moves['3side']:
            valid_moves_list.append(move)

        return valid_moves_list

    def check_for_liberty(self, board, i, j, color):
        stack = [(i, j)]
        visited = set()
        while stack:
            top_node = stack.pop()
            visited.add(top_node)
            for k in range(len(X_CHANGES)):
                new_i = top_node[0] + X_CHANGES[k]
                new_j = top_node[1] + Y_CHANGES[k]
                if 0 <= new_i < BOARD_SIZE and 0 <= new_j < BOARD_SIZE:
                    if (new_i, new_j) in visited:
                        continue
                    elif board[new_i][new_j] == UNOCCUPIED:
                        return True
                    elif board[new_i][new_j] == color and (new_i, new_j) not in visited:
                        stack.append((new_i, new_j))
        return False

    def get_opponent_color(self, color):
        return WHITE if color == BLACK else BLACK

    def check_for_ko(self, i, j):
        if self.previous_board[i][j] != self.player_color:
            return False
        new_board = copy.deepcopy(self.current_board)
        new_board[i][j] = self.player_color
        opponent_move_i, opponent_move_j = self.get_opponent_move()
        for k in range(len(X_CHANGES)):
            new_i = i + X_CHANGES[k]
            new_j = j + Y_CHANGES[k]
            if new_i == opponent_move_i and new_j == opponent_move_j:
                if not self.check_for_liberty(new_board, new_i, new_j, self.opponent_color):
                    self.remove_group(new_board, new_i, new_j, self.opponent_color)
        return np.array_equal(new_board, self.previous_board)

    def get_opponent_move(self):
        if np.array_equal(self.current_board, self.previous_board):
            return None
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if self.current_board[i][j] != self.previous_board[i][j] and self.current_board[i][j] != UNOCCUPIED:
                    return i, j

    def remove_group(self, board, i, j, color):
        stack = [(i, j)]
        visited = set()
        while stack:
            top_node = stack.pop()
            visited.add(top_node)
            board[top_node[0]][top_node[1]] = UNOCCUPIED
            for k in range(len(X_CHANGES)):
                new_i = top_node[0] + X_CHANGES[k]
                new_j = top_node[1] + Y_CHANGES[k]
                if 0 <= new_i < BOARD_SIZE and 0 <= new_j < BOARD_SIZE:
                    if (new_i, new_j) in visited:
                        continue
                    elif board[new_i][new_j] == color:
                        stack.append((new_i, new_j))
        return board

    def check_repetition(self, board):
        # Check for super-ko repetition using a transposition table
        state_key = tuple(tuple(row) for row in board)
        if state_key in self.transposition_table:
            return True
        self.transposition_table[state_key] = True
        return False

# Function to read input from the input file
def read_input(input_file_name=INPUT_FILE_NAME):
    with open(input_file_name) as input_file:
        input_file_lines = [line.strip() for line in input_file.readlines()]
        player_color = int(input_file_lines[0])
        previous_board = np.zeros((BOARD_SIZE, BOARD_SIZE), dtype=int)
        current_board = np.zeros((BOARD_SIZE, BOARD_SIZE), dtype=int)
        for i in range(1, 6):
            for j in range(len(input_file_lines[i])):
                previous_board[i - 1][j] = input_file_lines[i][j]
        for i in range(6, 11):
            for j in range(len(input_file_lines[i])):
                current_board[i - 6][j] = input_file_lines[i][j]
        return player_color, previous_board, current_board

# Function to write the output move to the output file
def write_output(next_move):
    with open(OUTPUT_FILE_NAME, 'w') as output_file:
        if next_move is None or next_move == (-1, -1):
            output_file.write('PASS')
        else:
            output_file.write(f'{next_move[0]},{next_move[1]}')

# Function to calculate the step number
def calculate_step_number(previous_board, current_board):
    previous_board_empty = True
    current_board_empty = True
    for i in range(BOARD_SIZE - 1):
        for j in range(BOARD_SIZE - 1):
            if previous_board[i][j] != UNOCCUPIED:
                previous_board_empty = False
                current_board_empty = False
                break
            elif current_board[i][j] != UNOCCUPIED:
                current_board_empty = False

    if previous_board_empty and current_board_empty:
        step_number = 0
    elif previous_board_empty and not current_board_empty:
        step_number = 1
    else:
        with open(STEP_NUMBER_FILE_NAME) as step_number_file:
            step_number = int(step_number_file.readline())
            step_number += 2

    with open(STEP_NUMBER_FILE_NAME, 'w') as step_number_file:
        step_number_file.write(f'{step_number}')

    return step_number

if __name__ == '__main__':
    # Main entry point of the program
    player_color, previous_board, current_board = read_input()
    step_number = calculate_step_number(previous_board, current_board)
    go_player = GoPlayer(player_color, previous_board, current_board)
    go_player.alpha_beta_search(4, 20, step_number)
